<h1><center> NASA Airathon - NO2 Track </center></h1>

### <center> Single day: extract OMI data for a specific location </center>

<div style="text-align: center"> 
    Dr. Sukanta Basu <br/> Associate Professor <br/> Delft University of Technology, The Netherlands <br/> Email: s.basu@tudelft.nl<br/> https://sites.google.com/view/sukantabasu/
</div>

#### Log

Last updated: 4th April, 2022

#### User instructions 

1. Provide the latitude (LATx) and longitude (LONx) of the location of interest 

2. Provide the filename (fOMI) containing OMI data for a specific date. 

3. This notebook will extract OMI data for the location of interest and save it in testOMI.csv file inside data/singleday/processed/test/OMI/ directory. 

4. Furthermore, the latitude, longitude information will be saved in testOBS.csv file inside data/singleday/processed/test/STN/ directory.

As an illustrative example, we use the centroid of the ZZ8JF grid (longitude: -117.3274620089189, latitude: 33.664840117597805). This information is available from the data/airathon/processed/grid_latlon.csv file. 

Also, we select August 24, 2021 as our case study date. The OMI filename is: 20210824T235959_omi_dl-la-tpe_0.nc. 

The OMI dataset is in hdf5 format. The NCKS tool, from the NCO package, is used to convert these files to netcdf format. Please use the following command for processing all the OMI files in a single directory: 

```
for FILE in *.he5; do ncks $FILE ${FILE%.he5}.nc; done
```

#### Load packages

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from pathlib import Path
import netCDF4

#### Directories

In [2]:
ROOT_DIR    = '../../'

#Only one day data is included in the repo_sukantabasu
OMI_DIR     = ROOT_DIR + 'data/singleday/raw/OMI/'

#Location of processed datasets
EXTDATA_DIR = ROOT_DIR + 'data/singleday/processed/'

#### Provide coordinates

In [3]:
LATx = 33.664840117597805
LONx = -117.3274620089189

#### Provide date

In [4]:
YR = 2021
MO = 8
DY = 24

#### OMI file name

In [5]:
fOMI = str(YR) + str(MO).zfill(2) + str(DY).zfill(2) + 'T235959_omi_dl-la-tpe_0.nc'
f    = OMI_DIR + fOMI

#### Compute nearest grid point

In [6]:
def nearestGridFAST(LAT,LON,LATx,LONx):
    
    dLAT = np.abs(LAT - LATx)
    dLON = np.abs(LON - LONx)
    
    dTOT = dLAT + dLON #taxi-cab distance
    
    #Interesting function; https://stackoverflow.com/questions/3230067/numpy-minimum-in-row-column-format
    r_min, c_min = np.unravel_index(dTOT.argmin(), dTOT.shape)
    
    return r_min, c_min

#### Grid structure of OMI data

In [7]:
LATomi = np.arange(-90, 90, 0.25)
LONomi = np.arange(-180, 180, 0.25)
LONomi2d, LATomi2d = np.meshgrid(LONomi, LATomi)

#### Extract OMI data 

In [8]:
dataOMI   = netCDF4.Dataset(f,'r')

NO2       = dataOMI.groups['HDFEOS'].groups['GRIDS'].groups['ColumnAmountNO2'].groups['Data Fields'].variables['ColumnAmountNO2CloudScreened'][:]            
NO2Tr     = dataOMI.groups['HDFEOS'].groups['GRIDS'].groups['ColumnAmountNO2'].groups['Data Fields'].variables['ColumnAmountNO2TropCloudScreened'][:]  

NO2       = NO2/1e15
NO2Tr     = NO2Tr/1e15

r_min, c_min = nearestGridFAST(LATomi2d,LONomi2d,LATx,LONx)

#Note: maskedArray.data takes care of the mask
#All the arrays are of size 720x1440
NO2_i     = NO2.data[r_min][c_min]
NO2Tr_i   = NO2Tr.data[r_min][c_min]

if NO2_i < 0:
    NO2_i = np.nan
if NO2Tr_i < 0:
    NO2Tr_i = np.nan

print((r_min,c_min,NO2_i,NO2Tr_i))


(495, 251, 6.7475924, 3.842983)


#### Save OMI data in a csv file

In [9]:
#Combine variables in an array
D = np.array([[NO2_i,NO2Tr_i]])

#Create pandas dataframes and save it as csv files   
df_omi = pd.DataFrame(data=D)
df_omi.columns = ['NO2_OMI','NO2Tr_OMI'] 
print(df_omi)

df_omi.to_csv(EXTDATA_DIR + 'test/OMI/' + 'testOMI.csv', index=False)

    NO2_OMI  NO2Tr_OMI
0  6.747592   3.842983


#### Save latitude & longitude data in a csv file

In [10]:
#Combine variables in an array
D = np.array([[LONx,LATx]])

#Create pandas dataframes and save it as csv files   
df_obs = pd.DataFrame(data=D)
df_obs.columns = ['longitude','latitude'] 
print(df_obs)

df_obs.to_csv(EXTDATA_DIR + 'test/STN/' + 'testOBS.csv', index=False)

    longitude  latitude
0 -117.327462  33.66484
